In [1]:
import pandas as pd

In [2]:
#load dataset
df=pd.read_table("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",sep=",",header=None)
df.columns=["Age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","num"]
df.head(4)

,Age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0


In [3]:
df["restecg"].unique()

array([2., 0., 1.])

In [4]:
#read_rules
def read_rules_from_csv(path):
    rules=pd.read_csv(path,index_col=0)
    return rules.T.to_dict(orient="list")

In [6]:
rules=read_rules_from_csv("dd.csv")

In [ ]:
def meta_wrap(a,b,method):
    #print(a,b,method)
    if method=="gt":
        return a>b
    if method=="neq":
        return a!=b
    if method=="eq":
        return a==b
    if method=="ge":
        return a>=b
    if method=="le":
        return a<=b
    if method=="lt":
        return a>b
    

In [ ]:
def apply_rules(row,rules,nr=0,decision_path=[]):
    rule=rules[nr]#node
    decision_path.append(nr)
    col=rule[0]
    signal=rule[1]
    value=rule[2]
    outcome_y=rule[3]
    outcome_y_type=rule[4]
    outcome_n=rule[5]
    outcome_n_type=rule[6]
    #print(data[col][0])
    if meta_wrap(row[col],value,signal):
        #print(outcome[0])
        if outcome_y_type=="final":
            #print("is string")
            return outcome_y
        else:
            return apply_rules(row,rules[int(outcome_y)],decision_path)
    else:
        #print(outcome[1])
        if outcome_n_type=="final":
            return outcome_n
        else:
            #print("outcome is nr")
            #print(rules[outcome[1]])
            return apply_rules(row,rules,int(outcome_n),decision_path)

In [ ]:
def create_decision(row,rules):
    #print(row)
    decision_path=[]
    return apply_rules(row,rules,decision_path=decision_path),decision_path

In [ ]:
create_decision(df.iloc[0,:],rules)

In [ ]:
df[["result","path"]]=df.apply(lambda x: create_decision(x,rules), axis=1, result_type='expand')

In [ ]:
def save_rules_to_csv(rules,path):
    dd=pd.DataFrame.from_dict(rules,orient='index')
    dd.columns=["Column","evaluator","comparasion","result_ok","result_ok_type","result_nok","result_nok_type"]
    dd.to_csv(path)

In [ ]:
save_rules_to_csv(rules,"rules.csv")

In [ ]:
df[["Age","sex","restecg","result","path"]]

In [ ]:
df["result"].unique()